In [1]:
from common import *

Using TensorFlow backend.


In [2]:
model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/w2v_model_300_w10'))
wv = model.wv

2017-07-27 12:22:01,373 : INFO : loading Word2Vec object from ../data/vectors/w2v_model_300_w10
2017-07-27 12:22:03,351 : INFO : loading wv recursively from ../data/vectors/w2v_model_300_w10.wv.* with mmap=None
2017-07-27 12:22:03,351 : INFO : loading syn0 from ../data/vectors/w2v_model_300_w10.wv.syn0.npy with mmap=None
2017-07-27 12:22:03,511 : INFO : setting ignored attribute syn0norm to None
2017-07-27 12:22:03,511 : INFO : loading syn1neg from ../data/vectors/w2v_model_300_w10.syn1neg.npy with mmap=None
2017-07-27 12:22:03,667 : INFO : setting ignored attribute cum_table to None
2017-07-27 12:22:03,667 : INFO : loaded ../data/vectors/w2v_model_300_w10


In [7]:
dictionary = corpora.Dictionary.load(join(DATA_FOLDER, 'pure.dict'))
tfidf = models.TfidfModel.load(join(DATA_FOLDER, 'tfidf_pure.model'))

2017-07-27 12:22:25,960 : INFO : loading Dictionary object from ../data/pure.dict
2017-07-27 12:22:27,311 : INFO : loaded ../data/pure.dict
2017-07-27 12:22:27,406 : INFO : loading TfidfModel object from ../data/tfidf_pure.model
2017-07-27 12:22:27,808 : INFO : loaded ../data/tfidf_pure.model


In [4]:
val_docs = glob(join(DATA_FOLDER, 'validate/*.txt'))

In [5]:
def match_query_document(query, doc, wv):
    query_words = set([w for w in query if w in wv])
    if len(query_words) == 0:
        return None
    
    doc_words = set(sum(doc, []))
    
    # fraction of valid words in query
    k = len(query_words)/len(query)
    

In [33]:
def sim_matrix(word_set1, word_set2, wv):
    l1 = list(word_set1)
    l2 = list(word_set2)
    wv1 = wv[l1]
    wv2 = wv[l2]
    
    arr = cosine_similarity(wv1, wv2)
    
    smat = pd.DataFrame(arr, index=l1, columns=l2)
    return smat
    

In [34]:
df = sim_matrix(['фосфонамида','металл','формула'], ['аралкильный','агент'], wv)
df

,аралкильный,агент
фосфонамида,0.513423,0.318413
металл,0.233888,0.205355
формула,0.343064,0.193441


In [42]:
np.amin(df, axis=0)

аралкильный    0.233888
агент          0.193441
dtype: float32

In [47]:
def wmd(smat, tfidf_weights):
    mins = np.amin(df, axis=0)
    return np.dot(mins, tfidf_weights)

In [31]:
def get_test_tfidf_weights(val_docs):
    tweights = {}
    for fname in tqdm(val_docs):
        with open(fname, 'r') as f:
            doc_text = f.read()
        doc_bow = dictionary.doc2bow(tokenize(doc_text))
        sorted_tfidf = sorted(tfidf[doc_bow], key=itemgetter(1), reverse=True)
        sorted_tfidf = pd.DataFrame([(dictionary[k],v) for k,v in sorted_tfidf],
                                   columns=['word', 'score'])
        
        name = path.splitext(basename(fname))[0]
        tweights[name] = sorted_tfidf
        
    return tweights

In [32]:
tweights = get_test_tfidf_weights(val_docs)

100%|██████████| 130/130 [00:00<00:00, 400.57it/s]


In [37]:
tweights['_01_98_2010120931A10010101RU']

,word,score
0,экономия,0.414882
1,кулинария,0.250506
2,электроэнергия,0.232453
3,томить,0.216007
4,блюдо,0.194750
5,старинный,0.181912
6,запиток,0.148888
7,никола,0.143980
8,кухня,0.133553
9,настаивать,0.131565


In [46]:
for key, doc_tfidf in enumerate(tqdm(tfidf[corpus])):
    docvec = doc_vec(doc_tfidf, wv, NWORDS)
    fname = all_docs[key]
    name = path.splitext(basename(fname))[0]
    train_names.append(name)
    train_vecs.append(docvec)

0.089510436356067646

In [ ]:
doc_dists = []

for fn in glob(join(DATA_FOLDER, 'documents/*.gz')):
    with GzipFile(fn, 'r') as myzip:
        text = myzip.read()
    docs = json.loads(text)
    for doc in docs:
        doc_words = set(sum(doc, []))
        dists = []
        for k, query in tweights.items():
            if len(query) == 0:
                dists.append(1)
                smat = sim_matrix(doc_words, query.word, wv)
                dist = wmd(smat, query.score)
                dists.append(dist)